# Imports

In [0]:
# inspect python version
import platform
print("Python Version", platform.python_version_tuple())

Python Version ('3', '6', '9')


In [0]:
# Check if in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print("IN_COLAB", IN_COLAB)

if IN_COLAB:
    # Mount google drive
    from google.colab import drive
    drive.mount('/content/drive')
    # drive.mount('/content/drive', force_remount=True)

    # update path to import modules from a folder in google drive.
    PROJECT_PATH = '/content/drive/My Drive/Unterlagen/MeinStudium/(6)_WiSe 2019-2020/Bachelor/BaBa/'
    sys.path.append(PROJECT_PATH)

    TORCH_DEVICE = 'cuda'
else:
    PROJECT_PATH = ''
    TORCH_DEVICE = 'cpu'

IN_COLAB True
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# torch
import torch
import torch.nn as nn
print("PyTorch Version", torch.__version__, "GPU", torch.cuda.get_device_name(), "GPU Capability",
      torch.cuda.get_device_capability(), "CUDA Version", torch.version.cuda,
      "CUDA available", torch.cuda.is_available())

PyTorch Version 1.4.0 GPU Tesla P100-PCIE-16GB GPU Capability (6, 0) CUDA Version 10.1 CUDA available True


In [0]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=967296b812dbcae43f3a2d29cec124d3dbc800cb4f78d7bf0c3f7dbbcac4eff0
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [0]:
# other modules
import os
import time
import math
import numpy as np
import PIL.Image
from ipywidgets import IntProgress
import imageio
import cv2
from collections import OrderedDict

import IPython.display
import IPython
print("IPython", IPython.__version__)

import matplotlib
print("matplotlib", matplotlib.__version__)
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
if IN_COLAB:
    %matplotlib inline
else:
    %matplotlib notebook

IPython 5.5.0
matplotlib 3.1.2
time: 2.13 s


In [0]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
        <p>{value} / {max}</p>
    """.format(value=value, max=max))

time: 2.42 ms


# Load data

In [0]:
originalImagesPath = os.path.join(PROJECT_PATH, "inputImages/Atlaszeder/")
scribbleImagesPath = os.path.join(PROJECT_PATH, "scribbleData/Atlaszeder/")
segmentedImagesPath = os.path.join(PROJECT_PATH, "segmentedData/Atlaszeder/")
validationImagesPath = os.path.join(PROJECT_PATH, "validationData/")

time: 2 ms


In [0]:
files = [f for f in os.listdir(originalImagesPath) if os.path.isfile(os.path.join(originalImagesPath, f))]
files.sort()
originalImagePaths = [os.path.join(originalImagesPath, filename) for filename in files]
print(originalImagePaths)

# get all scribble files related to originalImageFilename
files = [f for f in os.listdir(scribbleImagesPath) if os.path.isfile(os.path.join(scribbleImagesPath, f))]
files.sort()
scribbleImagePaths = [os.path.join(scribbleImagesPath, filename) for filename in files]
print(scribbleImagePaths)

# read images
colorMatsOriginal = [imageio.imread(path) for path in originalImagePaths]
scribbleMatsOriginalWithAlpha = [imageio.imread(path) for path in scribbleImagePaths]
scribbleMatsOriginal = [mat[..., :3] for mat in scribbleMatsOriginalWithAlpha]  # get rid of alpha channel

# scale down
newSize = (729, 486)
colorMats = [cv2.resize(mat, dsize=newSize) for mat in colorMatsOriginal]
scribbleMatsOriginalWithAlpha = [cv2.resize(mat, dsize=newSize) for mat in scribbleMatsOriginalWithAlpha]
scribbleMats = [cv2.resize(mat, dsize=newSize) for mat in scribbleMatsOriginal]

['/content/drive/My Drive/Unterlagen/MeinStudium/(6)_WiSe 2019-2020/Bachelor/BaBa/inputImages/Atlaszeder/DJI_0164.JPG', '/content/drive/My Drive/Unterlagen/MeinStudium/(6)_WiSe 2019-2020/Bachelor/BaBa/inputImages/Atlaszeder/DJI_0165.JPG', '/content/drive/My Drive/Unterlagen/MeinStudium/(6)_WiSe 2019-2020/Bachelor/BaBa/inputImages/Atlaszeder/DJI_0166.JPG', '/content/drive/My Drive/Unterlagen/MeinStudium/(6)_WiSe 2019-2020/Bachelor/BaBa/inputImages/Atlaszeder/DJI_0167.JPG', '/content/drive/My Drive/Unterlagen/MeinStudium/(6)_WiSe 2019-2020/Bachelor/BaBa/inputImages/Atlaszeder/DJI_0170.JPG', '/content/drive/My Drive/Unterlagen/MeinStudium/(6)_WiSe 2019-2020/Bachelor/BaBa/inputImages/Atlaszeder/DJI_0171.JPG', '/content/drive/My Drive/Unterlagen/MeinStudium/(6)_WiSe 2019-2020/Bachelor/BaBa/inputImages/Atlaszeder/DJI_0172.JPG', '/content/drive/My Drive/Unterlagen/MeinStudium/(6)_WiSe 2019-2020/Bachelor/BaBa/inputImages/Atlaszeder/DJI_0173.JPG', '/content/drive/My Drive/Unterlagen/MeinStudium

# Prepare train data

In [0]:
def prepareColorData(colorMat_):
    # normalize colorMat_ and add padding
    colorMat_ = np.divide(colorMat_, 255.0)
    colorMat_ = np.moveaxis(colorMat_, 2, 0)  # fit to Network requirements NxCxHxW later    
    #print("colorMat_", colorMat_.shape)
    return colorMat_

def prepareScribbleData(scribbleMat_):
    scribbleMat_ = np.delete(scribbleMat_, 1, 2)  # 1(green) 2(axis for colors)
    scribbleMat_ = np.minimum(scribbleMat_, 1)  # interval [0, 1]
    scribbleMat_ = np.moveaxis(scribbleMat_, 2, 0)  # fit to Network requirements NxCxHxW later
    scribbleMat_ = np.where(np.logical_and(scribbleMat_[0] == 0, scribbleMat_[1] == 0), np.full(scribbleMat_[1].shape, 0.5), scribbleMat_[0])
    #print("scribbleMat_", scribbleMat_.shape)
    return scribbleMat_

#coordMat = np.indices((imageHeight, imageWidth)).transpose((1, 2, 0)).astype(np.float32)
#coordMat[:, :, 0] = coordMat[:, :, 0] / imageHeight
#coordMat[:, :, 1] = coordMat[:, :, 1] / imageWidth
#print("coordMat", coordMat.shape)


time: 10.3 ms


In [0]:
xs_Color = [prepareColorData(mat) for mat in colorMats]
ys = [prepareScribbleData(mat) for mat in scribbleMats]
xs_Color = np.array(xs_Color)
ys = np.array(ys)
print("xs_Color shape:", xs_Color.shape)
print("ys shape:", ys.shape)

# shuffle
permutation = np.random.permutation(ys.shape[0])
xs_Color = xs_Color[permutation]
ys = ys[permutation]

# transform to torch tensors
tensor_xs_Color = torch.from_numpy(xs_Color).float()
tensor_ys = torch.from_numpy(ys).float()

tensor_ys = tensor_ys.unsqueeze(1)
print("tensor_ys shape unsqueezed:", tensor_ys.shape)

# split in data sets
tensor_xs_train_Color = tensor_xs_Color[:80, ...]
tensor_xs_test_Color = tensor_xs_Color[80:100, ...]
tensor_xs_eval_Color = tensor_xs_Color[100:, ...]
tensor_ys_train = tensor_ys[:80, ...]
tensor_ys_test = tensor_ys[80:100, ...]
tensor_ys_eval = tensor_ys[100:, ...]


train_dataset = torch.utils.data.TensorDataset(tensor_xs_train_Color, tensor_ys_train) # create datset
params = {'batch_size': 5,
          'shuffle': True,
          'num_workers': 6}
train_dataloader = torch.utils.data.DataLoader(train_dataset, **params) # create dataloader

xs_Color shape: (101, 3, 486, 729)
ys shape: (101, 486, 729)
tensor_ys shape unsqueezed: torch.Size([101, 1, 486, 729])
time: 2.06 s


# Model

In [0]:
class MyUNet(nn.Module):
    def __init__(self):
        super(MyUNet, self).__init__()
        
        in_channels=3
        out_channels=1
        features = 32

        self.enc1 = MyUNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=3)
        
        self.enc2 = MyUNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=3)
        
        self.enc3 = MyUNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=3)
        

        self.bottleneck = MyUNet._block(features * 4, features * 8, name="bottleneck")


        self.upconv3 = nn.ConvTranspose2d(features * 8, features * 4, kernel_size=3, stride=3)
        self.dec3 = MyUNet._block((features * 4) * 2, features * 4, name="dec3")

        self.upconv2 = nn.ConvTranspose2d(features * 4, features * 2, kernel_size=3, stride=3)
        self.dec2 = MyUNet._block((features * 2) * 2, features * 2, name="dec2")
        
        self.upconv1 = nn.ConvTranspose2d(features * 2, features, kernel_size=3, stride=3)
        self.dec1 = MyUNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(in_channels=features, out_channels=1, kernel_size=1)

    def forward(self, x):
        save1 = self.enc1(x)
        save2 = self.enc2(self.pool1(save1))
        save3 = self.enc3(self.pool2(save2))


        bottleneck = self.bottleneck(self.pool3(save3))


        x = self.upconv3(bottleneck)
        x = torch.cat((x, save3), dim=1)
        x = self.dec3(x)

        x = self.upconv2(x)
        x = torch.cat((x, save2), dim=1)
        x = self.dec2(x)
        
        x = self.upconv1(x)
        x = torch.cat((x, save1), dim=1)
        x = self.dec1(x)
        
        return torch.sigmoid(self.conv(x))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(OrderedDict([(name + "conv1", nn.Conv2d(in_channels=in_channels,
                                                                     out_channels=features,
                                                                     kernel_size=3,
                                                                     padding=1,
                                                                     bias=False)),
                                          (name + "norm1", nn.BatchNorm2d(num_features=features)),
                                          (name + "relu1", nn.ReLU(inplace=True)),
                                          (name + "conv2", nn.Conv2d(in_channels=features,
                                                                     out_channels=features,
                                                                     kernel_size=3,
                                                                     padding=1,
                                                                     bias=False)),
                                          (name + "norm2", nn.BatchNorm2d(num_features=features)),
                                          (name + "relu2", nn.ReLU(inplace=True))]))

time: 66.8 ms


# Train

In [29]:
net = MyUNet()
net.to(TORCH_DEVICE)

optimizer = torch.optim.Adadelta(net.parameters())#, weight_decay=0.1)
criterion = nn.BCELoss()

bar = display(progress(0, 100), display_id=True)

epochs = 45
for e in range(epochs):
    print("Epoch", e, end="  ")
    runningLoss = 0

    batchSize = len(train_dataloader)
    batchCount = 0
    bar.update(progress(batchCount, batchSize))

    for batchX, batchY in train_dataloader:
        optimizer.zero_grad()  # zero the gradient buffers
        batchX, batchY = batchX.to(TORCH_DEVICE), batchY.to(TORCH_DEVICE)
        batchY_pred = net(batchX)
        with torch.no_grad():
            batchY_this = torch.where(batchY == 0.5, batchY_pred, batchY)
        #print(batchY_pred.shape, batchY_this.shape)
        thisLoss = criterion(batchY_pred, batchY_this)
        thisLoss.backward()
        optimizer.step()
        runningLoss += thisLoss.item()

        batchCount += 1
        bar.update(progress(batchCount, batchSize))

    
    print("Loss", runningLoss, end="  ")

    # -- Test
    with torch.no_grad():
        ys_test_pred = net(tensor_xs_test_Color.to(TORCH_DEVICE)).cpu()
        testY_this = torch.where(tensor_ys_test == 0.5, ys_test_pred, tensor_ys_test)
    testLoss = criterion(ys_test_pred, testY_this)
    print("Test_Loss", testLoss.item())


Epoch 0  Loss 10.782551765441895  Test_Loss 0.6553146243095398
Epoch 1  Loss 9.988764524459839  Test_Loss 0.588512122631073
Epoch 2  Loss 8.859345614910126  Test_Loss 0.5152973532676697
Epoch 3  Loss 7.779045343399048  Test_Loss 0.4553791582584381
Epoch 4  Loss 6.841546028852463  Test_Loss 0.400061696767807
Epoch 5  Loss 6.0477874875068665  Test_Loss 0.35474079847335815
Epoch 6  Loss 5.385861903429031  Test_Loss 0.3184434771537781
Epoch 7  Loss 4.851417541503906  Test_Loss 0.28901180624961853
Epoch 8  Loss 4.425858914852142  Test_Loss 0.2641567587852478
Epoch 9  Loss 4.071138605475426  Test_Loss 0.24532988667488098
Epoch 10  Loss 3.8026131242513657  Test_Loss 0.22922194004058838
Epoch 11  Loss 3.57095730304718  Test_Loss 0.2166733294725418
Epoch 12  Loss 3.3917938619852066  Test_Loss 0.20573770999908447
Epoch 13  Loss 3.2188592106103897  Test_Loss 0.1975853592157364
Epoch 14  Loss 3.0930435210466385  Test_Loss 0.18891997635364532
Epoch 15  Loss 2.9618814289569855  Test_Loss 0.183315888

# Inference

In [30]:
net.train(False)
with torch.no_grad():
    ys_eval_Color = net(tensor_xs_eval_Color.to(TORCH_DEVICE))
    
ys_eval_Color = ys_eval_Color.cpu().squeeze(0).numpy()

time: 17.4 ms


# Display and Validate

In [0]:
def createImageFromYs(ys):
    #resultImage = ys[:, 0]
    resultImage = ys * 255
    #resultImage = np.moveaxis(resultImage, 0, 2)
    resultImage = resultImage.squeeze(0)
    resultImage = resultImage.astype(np.uint8, order='C', casting='unsafe')
    print("resultImage", resultImage.shape)
    return resultImage

time: 2.94 ms


In [31]:
fig, axs = plt.subplots(2, 2, figsize=(30, 2 * 10))

resultImage = createImageFromYs(ys_eval_Color)
resultImageThresh = cv2.threshold(resultImage, 127, 255, cv2.THRESH_BINARY)[1]
axs[0, 0].imshow(resultImage, cmap='gray', vmin=0, vmax=255)
axs[0, 1].imshow(resultImageThresh, cmap='gray', vmin=0, vmax=255)

axs[1, 0].imshow(np.moveaxis(tensor_xs_eval_Color.numpy().squeeze(0), 0, 2), vmin=0, vmax=255)
axs[1, 1].imshow(tensor_ys_eval.squeeze(0).squeeze(0).numpy() * 255, cmap='gray', vmin=0, vmax=255)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [0]:
# Info
print("shape: ", resultImageThresh.shape, resultImageThresh.size)
print("result 1", (resultImageThresh == 255).sum())
print("valida 1", (validationMat == 255).sum())
print("result 1 without valida 1", (resultImageThresh > validationMat).sum())
print("valida 1 without result 1", (validationMat > resultImageThresh).sum())

# Validation with ground truth mask
validationAccuracy = float((resultImageThresh == validationMat).sum()) / validationMat.size
print("validationAccuracy", validationAccuracy)

print()
print("Hit (intersect): ", (resultImageThresh == 255).sum() - (resultImageThresh > validationMat).sum())
print("Miss:            ", (validationMat > resultImageThresh).sum())
print("FalseAlarm:      ", (resultImageThresh > validationMat).sum())
print("CorrectRejection:", (resultImageThresh == validationMat).sum() - ((resultImageThresh == 255).sum() - (resultImageThresh > validationMat).sum()))
print()

# Intersection Over Union
intersection = (resultImageThresh == 255).sum() - (resultImageThresh > validationMat).sum()
print("Intersection", intersection)
union = (resultImageThresh == 255).sum() + (validationMat == 255).sum() - intersection
print("Union", union)
iou = float(intersection) / float(union)
print("Intersection Over Union", iou)

shape:  (364, 547) 199108
result 1 4580
valida 1 7514
result 1 without valida 1 1896
valida 1 without result 1 4830
validationAccuracy 0.9662193382485887

Hit (intersect):  2684
Miss:             4830
FalseAlarm:       1896
CorrectRejection: 189698

Intersection 2684
Union 9410
Intersection Over Union 0.28522848034006376
time: 29.9 ms


# Save

In [0]:
#message = "onlyColor5x16x16Patch_0.01ortsgew_20StrideInference_wenig daten aber 200 epochen"
#message = "onlyColor5x16x16Patch_10x10imageReduction_8epochs"
#message = "ColorCoordinates5x16x16_10x10imageReduction_coordsWeighting" + str(coordsWeighting) + "_8epochs"
#message = "onlyCoordinates_10x10imageReduction"
#message = "onlyColors_10x10imageReduction"
message = "combinedColorsCoordinates_10x10imageReduction"
timestr = time.strftime("%Y%m%d_%H%M%S")
segmentedImageFilename = timestr + "_" + originalImageFilename[:-4] + "_" + message + ".png"
segmentedImagePath = os.path.join(segmentedImagesPath, segmentedImageFilename)
print(imageio.imwrite(segmentedImagePath, resultImage))
print(segmentedImagePath)